In [80]:
!python -m pip install -U plotly
!python -m pip install -U gensim
!python -m pip install -U kneed
!python -m pip install -U scikit-learn
!python -m pip install -U pyLDAvis
!python -m pip install -U torchvision 

In [81]:
import numpy as np
import pandas as pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
import pyLDAvis

import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans
from gensim.matutils import corpus2csc
import torch
import torchvision



In [82]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import matplotlib.pyplot as plt 

#!pip install kneed
from kneed import KneeLocator

from sklearn.cluster import KMeans 
from scipy.cluster import hierarchy 
from sklearn.cluster import AgglomerativeClustering 
from sklearn.mixture import GaussianMixture

from gensim.models.doc2vec import Doc2Vec

from gensim.models.coherencemodel import CoherenceModel



In [90]:
from google.colab import files
files.upload()

Saving measures.py to measures (6).py


{'measures.py': b'from typing import List\r\nfrom matplotlib.pyplot import cohere\r\nimport pandas as pd\r\nimport numpy as np\r\nfrom gensim.models import LsiModel\r\nfrom gensim.models.coherencemodel import CoherenceModel\r\nfrom sklearn.metrics import cohen_kappa_score, adjusted_rand_score, silhouette_score\r\n\r\n"""\r\ncalculate the best amount of topics for LSI model\r\n"""\r\ndef best_num_topic(corpus, dictionary, max_n_topic)->List:\r\n    np.random.seed(0)\r\n    results = []\r\n\r\n    for t in range(2, max_n_topic):\r\n        lsi_model = LsiModel(corpus, id2word=dictionary, num_topics=t)\r\n        corpus_lsi = lsi_model[corpus]\r\n\r\n        cm = CoherenceModel(model=lsi_model, corpus=corpus_lsi, coherence=\'u_mass\')\r\n        score = cm.get_coherence()\r\n        tup = t, score\r\n        results.append(tup)\r\n\r\n    results = pd.DataFrame(results, columns=[\'topic\', \'score\'])\r\n    # lowest score means the best\r\n    s = pd.Series(results.score.values, index=re

In [93]:
import measures
from measures import get_coherence
from measures import get_silhouette_score


In [57]:
import plotly.io as pio
pio.renderers.default = "notebook"

In [71]:
def make_SVDtsne_2d(df, label=None, books=('a','b','c','d','e','f','g')):
  reduced = TruncatedSVD(n_components=7, random_state=0).fit_transform(df)

  tsne_results = TSNE(n_components=2, 
                      perplexity=40, 
                      init='pca', # globally more stable to initialization
                      learning_rate = 'auto',
                      n_iter=1000, 
                      random_state=0).fit_transform(reduced)

  df = pd.DataFrame()
  df['tsne-2d-one'] = tsne_results[:,0]
  df['tsne-2d-two'] = tsne_results[:,1]
  df['label_True'] = df_out['label'].loc[df_out['label'].isin(books)].reset_index(drop=True)
  print(df)
  if label is not None:
    dic = dict(enumerate(df_out['label'].loc[df_out['label'].isin(books)].unique()))
    lbl = list(label)
    for i, l in zip(range(0,len(lbl)), lbl):
      try:
        lbl[i] = dic[l].upper()
      except:
        lbl[i] = 'Outlier'
    df['label_Pred'] = lbl
  
  print(df)
  if label is None: fig = px.scatter(df, x='tsne-2d-one', y='tsne-2d-two', color="label_True", symbol="label_True", width=1200, height=900)
  if label is not None: fig = px.scatter(df, x='tsne-2d-one', y='tsne-2d-two', color="label_Pred", symbol="label_True", width=1200, height=900)

  fig.show()


In [72]:
def make_tsne_2d(df, label=None, books=('a','b','c','d','e','f','g')):
    # need to read raw our.csv first
    # df must be vectorized DataFrame
    
    tsne = TSNE(n_components=2, perplexity=40, n_iter=1000, random_state=0)
    tsne_results = tsne.fit_transform(df)
    
    df = pd.DataFrame()
    df['tsne-2d-one'] = tsne_results[:,0]
    df['tsne-2d-two'] = tsne_results[:,1]
    df['label_True'] = df_out['label'].loc[df_out['label'].isin(books)].reset_index(drop=True)
    if label is not None:
      dic = dict(enumerate(df_out['label'].unique()))
      lbl = list(label)
      for i, l in zip(range(0,len(lbl)), lbl):
        try:
          lbl[i] = dic[l].upper()
        except:
          lbl[i] = 'Outlier'
      df['label_Pred'] = lbl
    
    if label is None: fig = px.scatter(df, x='tsne-2d-one', y='tsne-2d-two', color="label_True", symbol="label_True", width=1200, height=900)
    if label is not None: fig = px.scatter(df, x='tsne-2d-one', y='tsne-2d-two', color="label_Pred", symbol="label_True", width=1200, height=900)
    fig.show()

In [73]:
def elbow_method(data, number):
    wcss = []
    for i in range(1, number+1):
        kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
        kmeans.fit(data)
        wcss.append(kmeans.inertia_)
    kn = KneeLocator(range(1, 10+1), wcss, curve='convex', direction='decreasing')

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=list(range(1, number+1)),
                            y=wcss))
    fig.add_vline(x=kn.knee, line_width=3, line_dash="dash", line_color="green")

    fig.update_layout(title='Elbow Method',
                      xaxis_title='Number of clusters',
                      yaxis_title='WCSS',
                      title_x=0.5,
                      height=500, 
                      width=800)
    fig.show()

In [92]:
def label_to_cluster_num(pred, k, books):
    pred_map = dict()
    offset = [(ord(label) - ord('a')) for label in books]

    for i in range(0, k):
        count = np.bincount(pred[i * 200 : i * 200 + 200])
        count.resize((1,k), refcheck=False) # resize row's dimension to 1xk
        count = count[0]
        index = np.where(count == max(count))[0][0]
        label =  chr(ord('a')+offset[i])
        pred_map[label] = index
        print(label + ":" + np.array2string(count) + "\t" + label + " mapped to " + str(index))

    return pred_map

In [94]:
def compare_predict(trans_data, data, modl, vidl = None,
  n_clusters=5,
  linkage='average',
  n_components = 2,
  covariance_type = 'diag',
  random_state = 0,
  books=('a','b','c','d','e','f','g')):

  X = trans_data[data['label'].isin(books)]

  # clustering parameters
  if 'n_clusters' in modl.get_params():
    modl.set_params(n_clusters = n_clusters)
  if 'linkage' in modl.get_params():
    modl.set_params(linkage = linkage)
  # em parameters
  if 'n_components' in modl.get_params():
    modl.set_params(n_components = n_components)
  if 'covariance_type' in modl.get_params():
    modl.set_params(covariance_type = covariance_type)
  if 'random_state' in modl.get_params():
    modl.set_params(random_state = random_state)

  pred = modl.fit_predict(X)

  pred_map = label_to_cluster_num(pred=pred, k=len(books), books= books)
  label_data_in_books = data[data['label'].isin(books)]['label'] # retrieve label rows that belongs to any of the books
  mapped_label = [pred_map[label] for label in label_data_in_books]

  if vidl is not None:
    if vidl == 'tsne':
      make_tsne_2d(X, pred, books)
    if vidl == 'SVDtsne':
      make_SVDtsne_2d(X, pred, books)

  return modl, pred, mapped_label

In [62]:
url = 'https://drive.google.com/file/d/1NmGTSp5CLiCTPlQgP614TvNNv1TXDewU/view?usp=sharin'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df_out = pd.read_csv(url)

df_out.drop('Unnamed: 0', axis=1, inplace=True)
df_out

,segment,label
0,act known circumst harm may accept gener test ...,a
1,aris whether represent part contract If contra...,a
2,section cite godbolt see F. N. B G fitzh abr d...,a
3,row bell c.c see treasur hidden anoth land D. ...,a
4,give trespass pretti nearli thing action case ...,a
...,...,...
1395,lamb chop fri tomato potato la crãªme rais bis...,g
1396,glass rhine wine IV glass water V glass champa...,g
1397,cover cloth set away cool place hour dinner ti...,g
1398,transpar veil egg cook white firm lift griddl ...,g


In [63]:
def lda_corpus(data):
  for i in range(len(data.index)):
    data['segment'][i] = data['segment'][i].split()
  data_lemmatized = data['segment'].tolist()
  # Create Dictionary 
  id2word = corpora.Dictionary(data_lemmatized)  
  # Create Corpus 
  texts = data_lemmatized  
  # Term Document Frequency 
  corpus = [id2word.doc2bow(text) for text in texts]  
  return corpus

In [64]:
corpus = lda_corpus(df_out)

In [101]:
model_save_name = 'lda_model.pt'
optimal_model = torch.load(model_save_name)

FileNotFoundError: ignored

In [65]:
df_lda = np.zeros(shape=(1400,4))
for i, row in enumerate(optimal_model[corpus]):
  for  j, (topic_num, prop_topic) in enumerate(row):
    df_lda[i, topic_num] = prop_topic

In [75]:
print(df_lda)

[[0.         0.         0.99924773 0.        ]
 [0.         0.06091685 0.93867427 0.        ]
 [0.         0.99915099 0.         0.        ]
 ...
 [0.         0.         0.         0.99902564]
 [0.         0.         0.         0.99902564]
 [0.         0.         0.         0.99902564]]


In [95]:
lda_kmeans5, lda_kmeans5_pred, lda_kmeans5_cluster_num = compare_predict(df_lda, df_out, KMeans(), n_clusters= 5)

a:[ 21 117   1   2  59   0   0]	a mapped to 1
b:[197   1   0   0   2   0   0]	b mapped to 0
c:[191   1   0   0   8   0   0]	c mapped to 0
d:[  0 188   1   2   9   0   0]	d mapped to 1
e:[  0   0 199   0   1   0   0]	e mapped to 2
f:[  0 199   0   0   1   0   0]	f mapped to 1
g:[  0   0   1 199   0   0   0]	g mapped to 3


In [96]:
lda_kmeans7, lda_kmeans7_pred, lda_kmeans7_cluster_num = compare_predict(df_lda, df_out, KMeans(), n_clusters= 7)

a:[ 1 87 14  2 41  9 46]	a mapped to 1
b:[  0   0 182   0   1  16   1]	b mapped to 2
c:[  0   1 136   0   5  58   0]	c mapped to 2
d:[  1 146   0   2   3   0  48]	d mapped to 1
e:[199   0   0   0   1   0   0]	e mapped to 0
f:[  0 196   0   0   1   0   3]	f mapped to 1
g:[  1   0   0 199   0   0   0]	g mapped to 3


In [97]:
lda_agglom5, lda_agglom5_pred, lda_agglom5_cluster_num = compare_predict(df_lda, df_out, AgglomerativeClustering(), n_clusters=5, linkage='ward')

a:[ 28   2 106   1  63   0   0]	a mapped to 2
b:[199   0   1   0   0   0   0]	b mapped to 0
c:[197   0   3   0   0   0   0]	c mapped to 0
d:[  0   2  73   1 124   0   0]	d mapped to 4
e:[  0   0   1 199   0   0   0]	e mapped to 3
f:[  0   0   4   0 196   0   0]	f mapped to 4
g:[  0 200   0   0   0   0   0]	g mapped to 1


In [98]:
lda_agglom7, lda_agglom7_pred, lda_agglom7_cluster_num = compare_predict(df_lda, df_out, AgglomerativeClustering(), n_clusters=7, linkage='ward')

a:[17  2 72  1 63 11 34]	a mapped to 2
b:[ 30   0   1   0   0 169   0]	b mapped to 5
c:[ 78   0   1   0   0 119   2]	c mapped to 5
d:[  0   2  71   1 124   0   2]	d mapped to 4
e:[  0   0   0 199   0   0   1]	e mapped to 3
f:[  0   0   3   0 196   0   1]	f mapped to 4
g:[  0 200   0   0   0   0   0]	g mapped to 1


In [99]:
lda_gmm5, lda_gmm5_pred, lda_gmm5_cluster_num = compare_predict(df_lda, df_out, GaussianMixture(), n_components=5)

a:[ 57   0  18   0 125   0   0]	a mapped to 4
b:[  0   0 178   0  22   0   0]	b mapped to 2
c:[  0   0 183   0  17   0   0]	c mapped to 2
d:[ 80   0   0   0 120   0   0]	d mapped to 4
e:[  0 184   0   0  16   0   0]	e mapped to 1
f:[189   0   0   0  11   0   0]	f mapped to 0
g:[  0   0   0 173  27   0   0]	g mapped to 3


In [100]:
lda_gmm7, lda_gmm7_pred, lda_gmm7_cluster_num = compare_predict(df_lda, df_out, GaussianMixture(), n_components=7)

a:[  9   0  10   0 116  57   8]	a mapped to 4
b:[  0   0 160   0  19   0  21]	b mapped to 2
c:[  0   0 114   0  76   0  10]	c mapped to 2
d:[105   0   0   0  10  80   5]	d mapped to 0
e:[  0 166   0   0   0   0  34]	e mapped to 1
f:[  9   0   0   0   1 189   1]	f mapped to 5
g:[  0   0   0 173   0   0  27]	g mapped to 3
